## Overview

This notebook will demonstrate how to query [Memgraph](https://memgraph.com/) populated with notional patient data. See `data/` with sample data.

## Background

In the realm of healthcare, every patient interaction forms the cornerstone of understanding their health journey. These interactions, termed as 'encounters', capture a snapshot of a patient's health status, the treatments provided, and the medical decisions taken during that particular point in time. By focusing on encounters, our knowledge base emphasizes:

* **Holistic Patient Journey**: Each encounter acts as a milestone, allowing us to piece together a complete and continuous narrative of a patient's healthcare experience.

* **Contextual Understanding**: An encounter-centric approach ensures that every diagnosis, test, or treatment is framed within the context of a specific patient interaction. This aids in understanding the rationale behind medical decisions and ensures that no data is viewed in isolation.

* **Scalability and Flexibility**: As healthcare systems grow and evolve, new types of encounters or details may emerge. Centering our database around encounters ensures that we can seamlessly integrate these new data points without disturbing the existing structure.

In essence, by adopting an encounter-centric design, we ensure that our database is not just a collection of data points but a dynamic representation of patient stories, told one encounter at a time.

## What kind of Questions can we ask of this Knowledge Base?

Some questions we can ask include:

* What diagnoses has a patient had?
* How many patients have had the same diagnoses?
* Which providers diagnosed those patients?

Let us walk through one-by-one to see how we can do this programmatically.

In [34]:
# standard libraries
from typing import List, Callable, Tuple, Any

# third party libraries
from mgclient import Cursor

# kbllm libraries
from kbllm.kb.context import MemgraphConnection


@MemgraphConnection
def query(cursor: Cursor, query: str, transform: Callable[[Tuple], Any]) -> List[Any]:
    """
    Executes a provided Cypher query and processes the results using a transformation function.

    Args:
        query (str): The Cypher query string to execute.
        transform (Callable[[Tuple], Any]): A function to process the raw results from the database.

    Returns:
        List[Any]: A list of processed results.
    """
    cursor.execute(query)
    results: Tuple = cursor.fetchall()

    return [transform(result) for result in results]

In [35]:
# kbllm libraries
from kbllm.kb.base import Diagnosis


def transform_to_diagnosis(result: Tuple) -> Diagnosis:
    """
    Transforms a tuple of results into a Diagnosis object.

    Args:
        result (Tuple): Raw results from the database.

    Returns:
        Diagnosis: Diagnosis object.
    """
    return Diagnosis(name=result[0], ICD10=result[1])

In [36]:
patient_name = "Alice Smith"

query_string = f"""
    MATCH (p:Patient {{name: "{patient_name}"}})-[:HAD_ENCOUNTER]->(e:Encounter)-[:HAS_DIAGNOSIS]->(d:Diagnosis)
    RETURN d.name AS Diagnosis_Name, d.ICD10 AS Diagnosis_Code
"""

In [37]:
alice_diagnoses = query(query_string, transform_to_diagnosis)
print(alice_diagnoses)

[Diagnosis(name='Hypertension', ICD10='I10'), Diagnosis(name='Eczema', ICD10='L30.9')]


Now we can programmatically interact with the diagnoses associated with Alice Smith.

Now let's explore how many patients have a given diagnosis.

In [9]:
# kbllm libraries
from kbllm.kb.base import Patient


def transform_to_patient(result: Tuple) -> Patient:
    """
    Transforms a tuple of results into a Patient object.

    Args:
        result (Tuple): Raw results from the database.

    Returns:
        Diagnosis: Patient object.
    """
    return Patient(name=result[0], MRN=result[1])

In [10]:
diagnoses_name = "Eczema"

query_string = f"""
    MATCH (d:Diagnosis {{name: "{diagnoses_name}"}})<-[:HAS_DIAGNOSIS]-(e:Encounter)<-[:HAD_ENCOUNTER]-(p:Patient)
    RETURN p.name AS Patient_Name, p.MRN AS MRN
"""

In [12]:
eczema_diagnoses = query(query_string, transform_to_patient)
print(eczema_diagnoses)

[Patient(name='Alice Smith', MRN='10001'), Patient(name='Charlie Williams', MRN='10003')]


Now we can programmatically interact with the patients that have been diagnosed with ezcema.

Finally, let's explore which providers diagnosed those patients with ezcema.

In [26]:
# kbllm libraries
from kbllm.kb.base import Provider


def transform_to_provider(result: Tuple) -> Provider:
    """
    Transforms a tuple of results into a Provider object.

    Args:
        result (Tuple): Raw results from the database.

    Returns:
        Diagnosis: Provider object.
    """
    return Provider(name=result[0], id=result[1], type=result[2], specialty=result[3])

In [32]:
diagnoses_name = "Eczema"

query_string = f"""
    MATCH (d:Diagnosis {{name: "{diagnoses_name}"}})<-[:HAS_DIAGNOSIS]-(e:Encounter)<-[:PROVIDED_ENCOUNTER]-(pr:Provider) 
    RETURN pr.name AS Provider_Name, pr.id AS Provider_ID, pr.type AS Type, pr.specialty AS Specialty
"""

In [33]:
eczema_providers = query(query_string, transform_to_provider)
print(eczema_providers)

[Provider(name='Dr. Mike Blue', id='4444444444', type='Doctor', specialty='Dermatology'), Provider(name='Dr. Cornelius', id='6666666666', type='Doctor', specialty='General Medicine')]


And there we have it, a way to programmatically interact with the Memgraph database. Feel free to try out your own queries. You'll need to define your own `Callable` to format the output.